In [116]:
import re
import pandas as pd
import pandas as pd
import numpy as np
import copy 

# 1. 데이터 전처리
### 1-1KB 국민카드 : 성별과 연령에 따른 소비자들의 선호 업종 분석

In [117]:
def agesexfunction(col):
    if col in ['01.남성_1.20대이하']:
        return "남성 20대 이하";
    if col in ['01.남성_2.30대']:
        return "남성 30대";
    if col in ['01.남성_3.40대']:
        return "남성 40대";
    if col in ['01.남성_4.50대']:
        return "남성 50대";
    if col in ['01.남성_5.60대']:
        return "남성 60대"
    if col in ['02.여성_1.20대이하']:
        return "여성 20대 이하";
    if col in ['02.여성_2.30대']:
        return "여성 30대";
    if col in ['02.여성_3.40대']:
        return "여성 40대";
    if col in ['02.여성_4.50대']:
        return "여성 50대";
    if col in ['02.여성_5.60대']:
        return "여성 60대"
    else:
        return 1000

In [118]:
def place(x):
    if x in ['국/탕/백반류','돼지고기','소고기','기타고기']:
        return '한식'
    
    if x in ['해산물']:
        return '일식/수산물'
    
    if x in ['닭고기']:
        return '닭/오리요리'
    
    if x in ['종합양식']:
        return '양식'
    
    if x in ['면요리']:
        return '분식'

In [119]:
def isbefore(value):
    if value in ['2019']:
        return "코로나 전"
    if value in ['2020', '2021']:
        return "코로나 후"
    else:
        return 1000

In [120]:
def findmin(value):
    if value in ['2019']:
        return "코로나 전"
    if value in ['2020', '2021']:
        return "코로나 후"
    else:
        return 1000

In [121]:
df1 = pd.read_csv('KB국민카드_금융빅데이터경진대회_데이터_raw2.csv')
dfbefore = df1[df1['기준년도']==2019]
dfb_group = dfbefore.groupby(by=['성별구분','연령대']).sum()[['매출건수']].reset_index()
dfafter = df1[df1['기준년도']==(2020|2021)]
dfa_group = dfafter.groupby(by=['성별구분','연령대']).sum()[['매출건수']].reset_index()

In [122]:
is_Seoul = df1['한글시도명'] == '서울특별시'
Seoul=df1[is_Seoul]
Seoul = Seoul.astype('string')##모든 데이터타입을 string으로
Seoul = Seoul.astype({'매출건수': 'int'})
Seoul['기준년월']=Seoul['기준년월'].str.zfill(2)
Seoul["날짜"] = Seoul["기준년도"].map(str)+"-"+Seoul["기준년월"]
pie_food = Seoul.groupby(['기준년도','성별구분','연령대','마케팅업종세분류명','마케팅업종세세분류명']).sum(['매출건수'])
pie_food = pie_food.reset_index()

pie_food=pie_food.astype({'성별구분' : 'string', '연령대' : 'string'})
cols=['성별구분', '연령대']
pie_food['성별연령'] = pie_food[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
pie_food.drop(columns=['성별구분', '연령대'], inplace=True)

pie_food['new성별연령']=pie_food['성별연령'].map(agesexfunction)
#TotalForeigner['생활지역권'] = TotalForeigner['행정동코드'].map(myfunction)
pie_food.drop(columns=['성별연령'], inplace=True)
pie_food = pie_food.astype('string')##모든 데이터타입을 string으로
pie_food = pie_food.astype({'매출건수': 'int'})
pie_food['중분류명']=pie_food['마케팅업종세분류명'].map(place)

In [144]:
## 서로 다른 csv 파일의 요식업종을 일치시키는 함수
def place(x):
    if x in ['국/탕/백반류','돼지고기','소고기','기타고기']:
        return '한식'
    
    if x in ['해산물']:
        return '일식/수산물'
    
    if x in ['닭고기']:
        return '닭/오리요리'
    
    if x in ['종합양식']:
        return '양식'
    
    if x in ['면요리']:
        return '분식'

### 1-2. 소상공인시장진흥공단_상가정보 : 서울시 내의 상가 분석

In [143]:
##행정동을 지역생활권으로 바꾸는 함수
def myfunction(x):
    #종로
    if re.search(x,'평창동, 부암동') is not None:
        return '평창.부암'
    if re.search(x,'종로1.2.3.4가동, 종로5.6가동, 혜화동, 이화동') is not None: #가동인지 그냥 동인지
        return '혜화.이화'
    if re.search(x,'창신1동 창신2동 창신3동, 숭인1동, 숭인2동부암동') is not None:
        return '창신.숭인'
    if re.search(x,' 청운효자동, 사직동, 삼청동, 무악동, 교남동, 가회동') is not None:
        return '청운효자.사직'
    
    #중구
    if re.search(x,'소공동, 회현동, 명동, 중림동') is not None:
        return '소공.회현'
    if re.search(x,'신당동, 황학동, 신당5동, 동화동, 청구동, 약수동, 다산동') is not None:
        return '신당.황학'
    if re.search(x,'광희동, 을지로동, 필동, 장충동') is not None:
        return '필동.장충'
    
    #용산구
    if re.search(x,'원효로1동, 원효로2동, 효창동, 용문동, 청파동') is not None:
        return '청파.원효'
    if re.search(x,'후암동, 용산2가동, 남영동, 이태원2동') is not None:
        return '후암.용산'
    if re.search(x,'한강로동, 서빙고동, 이촌1동, 이촌2동') is not None:
        return '이촌.한강'
    if re.search(x,'한남동, 이태원1동, 보광동') is not None:
        return '한남'
    
    
    #성동구
    if re.search(x,'성수1가1동 성수1가2동, 성수2가1동, 성수2가3동, 송정동') is not None:
        return '성수'
    if re.search(x,'금호1가동 금호2가동 금호3가동 금호4가동, 옥수동 금호2.3가동') is not None:
        return '금호.옥수 '
    if re.search(x,'마장동, 사근동, 용답동') is not None:
        return '마장.용답'
    if re.search(x,'왕십리도선동, 왕십리2동, 행당1동, 행당2동, 응봉동') is not None:
        return '왕십리.행당 '
    
    #강북구
    if re.search(x,'번1동 번2동 번3동') is not None:
        return '번동'
    if re.search(x,'삼각산동, 송천동, 송중동') is not None:
        return '삼각산'
    if re.search(x,'우이동, 인수동, 수유2동 수유3동') is not None:
        return '수유'
    if re.search(x,'수유1동, 삼양동, 미아동') is not None:
        return '미아'
    
    #중랑구
    if re.search(x,'묵1동 묵2동, 중화1동 중화2동, 상봉1동 상봉2동') is not None:
        return '중화.묵동'
    if re.search(x,'신내1동 신내2동, 망우본동') is not None:
        return '신내.망우'
    if re.search(x,'면목2동 면목3동 면목4동 면목5동, 면목7동 면목8동, 면목본동, 망우3동 면목3,8동') is not None:
        return '면목'
    
    #성북구
    if re.search(x,' 정릉1동 정릉2동 정릉3동 정릉4동') is not None:
        return '정릉'
    if re.search(x,'성북동, 삼선동, 동선동, 돈암2동, 안암동, 보문동') is not None:
        return '성북.동선'
    if re.search(x,'장위1동 장위2동 장위3동, 석관동') is not None:
        return '장위.석관'
    if re.search(x,'돈암1동, 종암동, 월곡2동') is not None:
        return '종암.월곡'
    if re.search(x,'길음1동,길음2동 , 월곡1동') is not None:
        return '길음'
    
    #동대문구
    if re.search(x,'장안1동 장안2동') is not None:
        return '장안'
    if re.search(x,'전농1동 전농2동, 답십리1동 답십리2동') is not None:
        return '전농.답십리'
    if re.search(x,'이문1동-이문2동, 회기동, 휘경1동 휘경2동') is not None:
        return '이문.휘경.회기 '
    if re.search(x,'청량리동, 제기동, 용신동') is not None:
        return '청량리'
    
    #노원구
    if re.search(x,'상계1동, 상계8동, 상계9동') is not None:
        return '마들'
    if re.search(x,'상계3.4동 상계5동') is not None:
        return '상계'
    if re.search(x,'상계2동 상계6.7동, 상계10동') is not None:
        return '노원'
    if re.search(x,'중계본동, 중계1동, 중계4동') is not None:
        return '중계'
    if re.search(x,'하계1동 하계2동 중계2.3동') is not None:
        return '하계'
    if re.search(x,'월계1동 월계2동-월계3동') is not None:
        return '월계'
    if re.search(x,'공릉1동 공릉2동') is not None:
        return '공릉'
    
    #광진구
    if re.search(x,'중곡1동 중곡2동 중곡3동 중곡4동') is not None:
        return '중곡'
    if re.search(x,'구의1동 구의2동 구의3동, 광장동') is not None:
        return '구의'
    if re.search(x,'군자동, 능동, 화양동') is not None:
        return '화양'
    if re.search(x,'자양1동 자양2동 자양3동-자양4동') is not None:
        return '자양'
    
    #도봉구
    if re.search(x,'도봉1동-도봉2동') is not None:
        return '도봉'
    if re.search(x,'방학1동 방학2동 방학3동') is not None:
        return '방학'
    if re.search(x,'쌍문1동-쌍문2동, 쌍문4동') is not None:
        return '쌍문'
    if re.search(x,'쌍문3동, 창2동-창3동') is not None:
        return '초안산'
    if re.search(x,'창1동, 창4동-창5동') is not None:
        return '창동'
    
    
    #은평구
    if re.search(x,'진관동') is not None:
        return '진관'
    if re.search(x,'갈현1동 갈현2동, 구산동, 역촌동') is not None:
        return '연신내'
    if re.search(x,'수색동, 증산동, 신사1동 신사2동') is not None:
        return '수색'
    if re.search(x,'불광1동 불광2동, 대조동') is not None:
        return '불광'
    if re.search(x,'응암1동 응암2동 응암3동, 녹번동') is not None:
        return '응암'
    
    #서대문구
    if re.search(x,'홍은1동홍은2동, 홍제1동 홍제2동 홍제3동') is not None:
        return '홍제'
    if re.search(x,'남가좌1동남가좌2동, 북가좌1동북가좌2동') is not None:
        return '가좌'
    if re.search(x,'천연동, 충현동, 북아현동') is not None:
        return '충청'
    if re.search(x,'연희동, 신촌동') is not None:
        return '신촌'
    
    #마포구
    if re.search(x,'상암동') is not None:
        return '상암'
    if re.search(x,'성산1동 성산2동, 망원1동망원2동') is not None:
        return '성산'
    if re.search(x,'연남동, 서교동, 합정동, 서강동') is not None:
        return '합정.서교'
    if re.search(x,'대흥동, 염리동, 아현동, 공덕동') is not None:
        return '아현'
    if re.search(x,'신수동, 용강동, 도화동') is not None:
        return '용강'
    
    #양천구
    if re.search(x,'신월1동, 신월3동, 신월5동') is not None:
        return '신월1'
    if re.search(x,'신월2동, 신월4동, 신월6동-신월7동') is not None:
        return '신월2'
    if re.search(x,'신정1동, 신정3동-신정4동') is not None:
        return '신정'
    if re.search(x,'목2동, 목3동, 목4동') is not None:
        return '목동1'
    if re.search(x,'목1동, 목5동, 신정2동, 신정6동 신정7동') is not None:
        return '목동2'
    
    #동작구
    if re.search(x,'노량진1동 노량진2동, 대방동') is not None:
        return '노량진'
    if re.search(x,'흑석동') is not None:
        return '흑석'
    if re.search(x,'신대방1동 신대방2동') is not None:
        return '신대방'
    if re.search(x,'상도1동 상도2동 상도3동 상도4동') is not None:
        return '상도'
    if re.search(x,'사당1동 사당2동 사당3동 사당4동 사당5동') is not None:
        return '사당'
    
    #금천구
    if re.search(x,'가산동') is not None:
        return '가산'
    if re.search(x,'독산1동 독산2동 독산3동 독산4동') is not None:
        return '독산'
    if re.search(x,'시흥1동 시흥2동 시흥3동 시흥4동 시흥5동') is not None:
        return '시흥'
    
    #영등포구
    if re.search(x,'여의도동 여의동') is not None:
        return '여의도'
    if re.search(x,'당산1동, 양평1동, 도림동, 문래동, 영등포동, 영등포본동') is not None:
        return '영등포'
    if re.search(x,'당산2동, 양평2동') is not None:
        return '당산'
    if re.search(x,'신길1동,신길2동 신길3동 신길4동 신길5동 신길6동 신길7동') is not None:
        return '신길'
    if re.search(x,'대림1동 대림2동 대림3동') is not None:
        return '대림'
    
    #구로구
    if re.search(x,'고척1동 고척2동, 개봉1동 개봉2동 개봉3동') is not None:
        return '고척.개봉'
    if re.search(x,'오류1동 오류2동, 수궁동') is not None:
        return '오류.수궁'
    
    ##3동!!!
    if re.search(x,'신도림동, 구로1동 구로2동, 구로5동') is not None:
        return '구로신도림'
    if re.search(x,'가리봉동, 구로2동, 구로4동 구로3동') is not None:
        return '구로디지털단지'
    
    #강서구
    if re.search(x,'공항동, 방화1동 방화2동 방화3동') is not None:
        return '공항.방화'
    if re.search(x,'등촌3동, 가양1동') is not None:
        return '마곡'
    if re.search(x,'염창동, 등촌1동, 가양2동 가양3동') is not None:
        return '염창'
    if re.search(x,'우장산동, 발산1동, 화곡3동') is not None:
        return '발산'
    if re.search(x,'화곡1동 화곡2동, 화곡4동, 화곡8동') is not None:
        return '화곡1'
    if re.search(x,'등촌2동, 화곡본동, 화곡6동') is not None:
        return '화곡2'
    
    #관악구
    if re.search(x,'청림동, 행운동, 낙성대동, 인헌동, 남현동') is not None:
        return '낙성대'
    if re.search(x,'신사동, 난향동, 조원동, 미성동, 난곡동') is not None:
        return '난곡'
    if re.search(x,'보라매동, 서원동, 신원동, 신림동') is not None:
        return '신림'
    if re.search(x,'중앙동, 은천동, 성현동, 청룡동') is not None:
        return '봉천'
    if re.search(x,'서림동, 대학동, 삼성동') is not None:
        return '대학'
    
    #서초구
    if re.search(x,'반포본동, 반포1동 반포2동 반포3동 반포4동, 잠원동') is not None:
        return '반포'
    if re.search(x,'방배본동, 방배1동 방배2동 방배3동 방배4동') is not None:
        return '방배'
    if re.search(x,'서초1동 서초2동 서초3동 서초4동') is not None:
        return '서초'
    if re.search(x,'양재1동 양재2동, 내곡동') is not None:
        return '양재'
    
    #강동구
    if re.search(x,'암사1동 암사2동 암사3동') is not None:
        return '암사'
    if re.search(x,'명일1동 명일2동, 고덕1동 고덕2동') is not None:
        return '명일.고덕'
    if re.search(x,'천호1동 천호2동 천호3동, 성내1동 성내2동 성내3동') is not None:
        return '천호.성내'
    if re.search(x,'강일동, 상일동') is not None:
        return '강일.상일'
    if re.search(x,'둔촌1동 둔촌2동, 길동') is not None:
        return '길동.둔촌'
    
    #강남구
    if re.search(x,'신사동, 압구정동, 청담동') is not None:
        return '압구정.청담'
    if re.search(x,'삼성1동 삼성2동') is not None:
        return '삼성'
    if re.search(x,'역삼1동 역삼2동, 논현1동 논현2동') is not None:
        return '역삼.논현 '
    if re.search(x,'도곡1동 도곡2동, 대치1동 대치2동, 대치4동') is not None:
        return '대치.도곡'
    if re.search(x,'개포1동 개포2동, 개포4동, 일원1동 일원2동') is not None:
        return '개포.일원'
    if re.search(x,'수서동, 세곡동, 일원본동') is not None:
        return '수서.세곡'
    
    #송파구 - 
    if re.search(x,'잠실본동, 잠실2동 잠실3동, 잠실7동') is not None:
        return '잠실1'
    if re.search(x,'잠실4동, 잠실6동, 풍납1동 풍납2동') is not None:
        return '잠실2'
    if re.search(x,'송파1동 송파2동, 방이1동 방이2동') is not None:
        return '송파'
    if re.search(x,'오륜동, 오금동, 가락본동, 가락2동') is not None:
        return '가락'
    if re.search(x,'삼전동, 석촌동, 가락1동') is not None:
        return '석촌'
    if re.search(x,'문정1동 문정2동, 장지동, 위례동') is not None:
        return '문정'    
    if re.search(x,'거여1동, 거여2동, 마천1동, 마천2동, 위례동') is not None:
        return '거여.마천'
    
    return x+'dd'

In [125]:
df2 = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_서울_202103.csv')
df2=df2[[ '상권업종대분류코드', '상권업종대분류명', '상권업종중분류명',  '상권업종소분류명', '표준산업분류명', 
        '시군구명','법정동코드', '법정동명', '대지구분명', '행정동명' ]]
df2.rename(columns = {'상권업종대분류코드' : '대분류코드','상권업종대분류명' : '대분류명',
                     '상권업종중분류명' : '중분류명','상권업종소분류명' : '소분류명',
                    '표준산업분류명' : '분류명',},inplace = True)
df2 = df2.loc[(df2['대분류명'] == '음식') &(df2['대분류코드']=='Q'),:] # 음식인 것들만
df2.reset_index(drop=True, inplace=True) #index정리
df2['생활지역권']=df2['행정동명'].map(myfunction)
whichregion=copy.deepcopy(df2[['행정동명', '생활지역권']])
df2.drop(columns=['대분류코드','대분류명','대지구분명','시군구명','법정동코드','법정동명','행정동명','분류명'],inplace = True)
onemenuSelect=copy.deepcopy(df2)

### 1-3. 서울생활인구 : 예비소비자 분석

In [145]:
##행정동 코드를 지역생활권으로 바꾸는 함수
def regionfunction(x):
    #종로
    if x in [1111056000, 1111055000]:
        return '평창.부암'
    if x in [1111061500, 1111063000, 1111064000, 1111065000]:
        return '혜화.이화'
    if x in [1111067000, 1111068000, 1111069000, 1111070000, 1111071000]:
        return '창신.숭인'
    if x in [1111051500, 1111053000, 1111054000, 1111057000, 1111058000, 1111060000]:
        return '청운효자.사직'
    
    #중구
    if x in [1114052000, 1114054000, 1114055000, 1114068000]:
        return '소공.회현'
    if x in [1114061500, 1114062500, 1114063500, 1114064500, 1114065000, 1114066500, 1114067000]:
        return '신당.황학'
    if x in [1114057000, 1114058000, 1114059000, 1114060500]:
        return '필동.장충'
    
    #용산구
    if x in [1117055500, 1117056000, 1117057000, 1117058000, 1117059000]:
        return '청파.원효'
    if x in [1117051000, 1117052000, 1117053000, 1117066000]:
        return '후암.용산'
    if x in [1117063000, 1117064000, 1117062500, 1117069000]:
        return '이촌.한강'
    if x in [1117065000, 1117068500, 1117070000]:
        return '한남'
    
    
    #성동구
    if x in [1120065000, 1120066000, 1120067000, 1120069000, 1120072000]:
        return '성수'
    if x in [1120059000, 1120061500, 1120062000, 1120064500]:
        return '금호.옥수 '
    if x in [1120054000, 1120055000, 1120079000]:
        return '마장.용답'
    if x in [1120052000, 1120053500, 1120056000, 1120057000, 1120058000]:
        return '왕십리.행당 '
    
    #강북구
    if x in [1130559000, 1130560000, 1130560600]: 
        return '번동'
    if x in [1130554500, 1130555500, 1130557500]:
        return '삼각산'
    if x in [1130562000, 11305635000, 1130564000, 1130566000,1130564500]:  
        return '수유'
    if x in [1130561000, 1130553400, 1130553500, 1130563000]: 
        return '미아'
    
    #중랑구
    if x in [1126058000, 1126059000, 1126060000, 1126061000, 1126062000, 1126063000]:
        return '중화.묵동'
    if x in [1126068000, 1126069000, 1126065500]:
        return '신내.망우'
    if x in [1126052000, 1126054000, 1126055000, 1126056500, 1126057000, 1126057500, 1126066000]:
        return '면목'
    
    #성북구
    if x in [1129062000, 1129063000, 1129064000, 1129065000]:
        return '정릉'
    if x in [1129052500, 1129055500, 1129057500, 1129059000, 1129060000, 1129061000]:
        return '성북.동선'
    if x in [1129076000, 1129077000, 1129078000, 1129081000]:
        return '장위.석관'
    if x in [1129058000, 1129070500, 1129072500]:
        return '종암.월곡'
    if x in [1129066000, 1129068500, 1129071500]:
        return '길음'
    
    #동대문구
    if x in [1123065000, 1123066000]:
        return '장안'
    if x in [1123056000, 1123057000, 1123060000, 1123061000]:
        return '전농.답십리'
    if x in [1123071000, 1123072000, 1123073000, 1123074000, 1123075000]:
        return '이문.휘경.회기 '
    if x in [1123070500, 1123053600, 1123054500]:
        return '청량리'
    
    #노원구
    if x in [1135063000, 1135070000, 1135071000]:
        return '마들'
    if x in [1135066500, 1135067000]:
        return '상계'
    if x in [1135064000, 1135069500, 1135072000]:
        return '노원'
    if x in [1135061900, 1135062100, 1135062400]:
        return '중계'
    if x in [1135061100, 1135061200, 1135062500]:
        return '하계'
    if x in [1135056000, 1135057000, 1135058000]:
        return '월계'
    if x in [1135059500, 1135060000]:
        return '공릉'
    
    #광진구
    if x in [1121574000, 1121575000, 1121576000, 1121577000]:
        return '중곡'
    if x in [1121585000, 1121586000, 1121587000, 1121581000]:
        return '구의'
    if x in [1121571000, 1121573000, 1121578000]:
        return '화양'
    if x in [1121582000, 1121583000, 1121584000, 1121584700]:
        return '자양'
    
    #도봉구
    if x in [1132052100, 1132052200]:
        return '도봉'
    if x in [1132069000, 1132070000, 1132071000]:
        return '방학'
    if x in [1132066000, 1132067000, 1132068100]:
        return '쌍문'
    if x in [1132068000, 1132051200, 1132051300]:
        return '초안산'
    if x in [1132051100, 1132051400, 1132051500]:
        return '창동'
    
    
    #은평구
    if x in [1138069000]:
        return '진관'
    if x in [1138055100, 1138055200, 1138056000, 1138062500]:
        return '연신내'
    if x in [1138063100, 1138063200, 1138064000, 1138065000]:
        return '수색'
    if x in [1138057000, 1138052000, 1138053000]:
        return '불광'
    if x in [1138058000, 1138059000, 1138060000, 1138051000]:
        return '응암'
    
    #서대문구
    if x in [1141062000, 1141064000, 1141065500, 1141066000, 1141068500]:
        return '홍제'
    if x in [1141069000, 1141070000, 1141071000, 1141072000]:
        return '가좌'
    if x in [1141052000, 1141055500, 1141056500]:
        return '충청'
    if x in [1141058500, 1141061500]:
        return '신촌'
    
    #마포구
    if x in [1144074000]:
        return '상암'
    if x in [1144072000, 1144073000, 1144069000, 1144070000]:
        return '성산'
    if x in [1144071000, 1144065500, 1144066000, 1144068000]:
        return '합정.서교'
    if x in [1144060000, 1144061000, 1144055500, 1144056500]:
        return '아현'
    if x in [1144063000, 1144058500, 1144059000]:
        return '용강'
    
    #양천구
    if x in [1147056000, 1147058000, 1147060000]:
        return '신월1'
    if x in [1147057000, 1147059000, 1147061000, 1147061100]:
        return '신월2'
    if x in [1147062000, 1147064000, 1147065000]:
        return '신정'
    if x in [1147052000, 1147053000, 1147054000]:
        return '목동1'
    if x in [1147051000, 1147055000, 1147063000, 1147067000, 1147068000]:
        return '목동2'
    
    #동작구
    if x in [1159051000, 1159052000, 1159066000]:
        return '노량진'
    if x in [1159060500]:
        return '흑석'
    if x in [1159067000, 1159068000]:
        return '신대방'
    if x in [1159053000, 1159054000, 1159055000, 1159056000]:
        return '상도'
    if x in [1159062000, 1159063000, 1159064000, 1159065000, 1159065100]:
        return '사당'
    
    #금천구
    if x in [1154551000]:
        return '가산'
    if x in [1154561000, 1154562000, 1154563000, 1154564000]:
        return '독산'
    if x in [1154567000, 1154568000, 1154569000, 1154570000, 1154571000]:
        return '시흥'
    
    #영등포구
    if x in [1156054000]:
        return '여의도'
    if x in [1156055000, 1156061000, 1156058500, 1156060500, 1156051500, 1156053500]:
        return '영등포'
    if x in [1156056000, 1156062000]:
        return '당산'
    if x in [1156063000, 1156065000, 1156066000, 1156067000, 1156068000, 1156069000]:
        return '신길'
    if x in [1156070000, 1156071000, 1156072000]:
        return '대림'
    
    #구로구
    if x in [1153072000, 1153073000, 1153074000, 1153075000, 1153076000]:
        return '고척.개봉'
    if x in [1153077000, 1153078000, 1153079000]:
        return '오류.수궁'
    
    ##3동!!!
    if x in [1153051000, 1153052000, 1153053000, 1153056000]:
        return '구로신도림'
    if x in [1153054000, 1153055000, 1153059500]:
        return '구로디지털단지'
    
    #강서구
    if x in [1150062000, 1150063000, 1150064000, 1150064100]:
        return '공항.방화'
    if x in [1150053500, 1150060300]:
        return '마곡'
    if x in [1150051000, 1150052000, 1150060400, 1150060500]:
        return '염창'
    if x in [1150061100, 1150061500, 1150056000]:
        return '발산'
    if x in [1150054000, 1150055000, 1150057000,  1150059300]:
        return '화곡1'
    if x in [1150053000, 1150059000, 1150059100]:
        return '화곡2'
    
    #관악구
    if x in [1162054500, 1162057500, 1162058500, 1162062500, 1162063000]:
        return '낙성대'
    if x in [1162068500, 1162071500, 1162072500, 1162076500, 1162077500]:
        return '난곡'
    if x in [1162052500, 1162064500, 1162065500, 1162069500]:
        return '신림'
    if x in [1162059500, 1162060500, 1162061500, 1162056500]:
        return '봉천'
    if x in [1162066500, 1162073500, 1162074500]:
        return '대학'
    
    #서초구
    if x in [1165054000, 1165055000, 1165056000, 1165057000, 1165058000, 1165058100]:
        return '반포'
    if x in [1165059000, 1165060000, 1165061000, 1165062000, 1165062100]:
        return '방배'
    if x in [1165051000, 1165052000, 1165053000, 1165053100]:
        return '서초'
    if x in [1165065100, 1165065200, 1165066000]:
        return '양재'
    
    #강동구
    if x in [1174057000, 1174058000, 1174059000]:
        return '암사'
    if x in [1174053000, 1174054000, 1174055000, 1174056000]:
        return '명일.고덕'
    if x in [1174060000, 1174061000, 1174062000, 1174064000, 1174065000, 1174066000]:
        return '천호.성내'
    if x in [1174051500, 1174052000]:
        return '강일.상일'
    if x in [1174068500, 1174069000, 1174070000]:
        return '길동.둔촌'
    
    #강남구
    if x in [1168051000, 1168054500, 1168056500]:
        return '압구정.청담'
    if x in [1168058000, 1168059000]:
        return '삼성'
    if x in [1168064000, 1168065000, 1168052100, 1168053100]:
        return '역삼.논현 '
    if x in [1168065500, 1168065600, 1168060000, 1168061000, 1168063000]:
        return '대치.도곡'
    if x in [1168066000, 1168067000, 1168069000, 1168073000, 1168074000]:
        return '개포.일원'
    if x in [1168070000, 1168075000, 1168072000]:
        return '수서.세곡'
    
    #송파구 - 
    if x in [1171072000, 1171067000, 1171068000, 1171065000]:
        return '잠실1'
    if x in [1171069000, 1171071000, 1171051000, 1171052000]:
        return '잠실2'
    if x in [1171058000, 1171059000, 1171056100, 1171056200]:
        return '송파'
    if x in [1171056600, 1171057000, 1171062000, 1171063200]:
        return '가락'
    if x in [1171060000, 1171061000, 1171063100]:
        return '석촌'
    if x in [1171064100, 1171064200, 1171064600, 1171064700]:
        return '문정'    
    if x in [1171053100, 1171053200, 1171054000, 1171055000, 1171064700]:
        return '거여.마천'
    
    return 0

In [148]:
##시간중 정수 자료형이 아닌 데이터를 을 정수형 시간으로 변환하는 함수
def number(time):
    if type(time) == 'integer': #in [0,1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19,20,21,22,23]:
        return time
    if time =='0': return 0
    if time in ['01','1']: return 1
    if time in ['02','2']: return 2
    if time in ['03','3']: return 3
    if time in ['04','4']: return 4
    if time in ['05','5']: return 5
    if time in ['06','6']: return 6
    if time in ['07','7']: return 7
    if time in ['08','8']: return 8
    if time in ['09','9']: return 9
    if time == '10': return 10
    if time == '11': return 11
    if time == '12': return 12
    if time == '13': return 13
    if time == '14': return 14
    if time == '15': return 15
    if time == '16': return 16
    if time == '17': return 17
    if time == '18': return 18
    if time == '19': return 19
    if time == '20': return 20
    if time == '21': return 21
    if time == '22': return 22
    if time == '23': return 23
    
    return 9999
    

### 1. 2019 생활인구

In [128]:
# 2019생활인구
import pandas as pd
Local201901=pd.read_csv(r'LOCAL_PEOPLE_DONG_201901.csv', error_bad_lines=False)
Local201901=Local201901.reset_index()
Local201901=Local201901.shift(periods=6,axis=1)
Local201901=Local201901.drop(columns=['level_0','level_1','level_2','level_3','level_4','level_5'])

Local201902=pd.read_csv(r'LOCAL_PEOPLE_DONG_201902.csv', engine='python', error_bad_lines=False,encoding='UTF8' )
Local201902=Local201902.reset_index()
Local201902=Local201902.shift(periods=1,axis=1)
Local201902=Local201902.drop(columns=['index'])

Local201903=pd.read_csv(r'LOCAL_PEOPLE_DONG_201903.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local201903=Local201903.reset_index()
Local201903=Local201903.shift(periods=6,axis=1)
Local201903=Local201903.drop(columns=['level_0','level_1','level_2','level_3','level_4','level_5'])

Local201904=pd.read_csv(r'LOCAL_PEOPLE_DONG_201904.csv',engine='python', error_bad_lines=False, encoding='UTF8')
Local201904=Local201904.reset_index()
Local201904=Local201904.shift(periods=1,axis=1)
Local201904=Local201904.drop(columns=['index'])

Local201905=pd.read_csv(r'LOCAL_PEOPLE_DONG_201905.csv',engine='python', error_bad_lines=False, encoding='UTF8')
Local201905=Local201905.reset_index()
Local201905=Local201905.shift(periods=1,axis=1)
Local201905=Local201905.drop(columns=['index'])

Local201906=pd.read_csv(r'LOCAL_PEOPLE_DONG_201906.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local201906=Local201906.reset_index()
Local201906=Local201906.shift(periods=1,axis=1)
Local201906=Local201906.drop(columns=['index'])

Local201907=pd.read_csv(r'LOCAL_PEOPLE_DONG_201907.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local201907=Local201907.reset_index()
Local201907=Local201907.shift(periods=1,axis=1)
Local201907=Local201907.drop(columns=['index'])

Local201908=pd.read_csv(r'LOCAL_PEOPLE_DONG_201908.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local201908=Local201908.reset_index()
Local201908=Local201908.shift(periods=1,axis=1)
Local201908=Local201908.drop(columns=['index'])

Local201909=pd.read_csv(r'LOCAL_PEOPLE_DONG_201909.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local201909=Local201909.reset_index()
Local201909=Local201909.shift(periods=1,axis=1)
Local201909=Local201909.drop(columns=['index'])

#인코딩 이상하게 된 것 같아요 info 하면 잘 ㅇ안나와서 나중에 다시 추가해야할듯..! S:encoding='cp949'
Local201910=pd.read_csv(r'LOCAL_PEOPLE_DONG_201910.csv', engine='python', error_bad_lines=False, encoding='cp949')
Local201910=Local201910.reset_index()
Local201910=Local201910.shift(periods=1,axis=1)
Local201910=Local201910.drop(columns=['index'])

Local201911=pd.read_csv(r'LOCAL_PEOPLE_DONG_201911.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local201911=Local201911.reset_index()
Local201911=Local201911.shift(periods=1,axis=1)
Local201911=Local201911.drop(columns=['index'])

Local201912=pd.read_csv(r'LOCAL_PEOPLE_DONG_201912.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local201912=Local201912.reset_index()
Local201912=Local201912.shift(periods=1,axis=1)
Local201912=Local201912.drop(columns=['index'])

Local2019 = pd.concat([Local201903,Local201904,Local201905,Local201906,Local201907,Local201908,Local201909, #Local201910,
                      Local201911,Local201912], ignore_index = True)

## Local2019 = Local2019[['기준일ID','시간대구분','행정동코드','총생활인구수']]
Local2019.행정동코드 = Local2019.행정동코드 * 100
Local2019.총생활인구수 = Local2019.총생활인구수 * 10000
Local2019 = Local2019.astype({'기준일ID':'string'})
#Local2020.총생활인구수
Local2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2798400 entries, 0 to 2798399
Data columns (total 32 columns):
 #   Column           Dtype  
---  ------           -----  
 0   기준일ID            string 
 1   시간대구분            int64  
 2   행정동코드            int64  
 3   총생활인구수           float64
 4   남자0세부터9세생활인구수    float64
 5   남자10세부터14세생활인구수  float64
 6   남자15세부터19세생활인구수  float64
 7   남자20세부터24세생활인구수  float64
 8   남자25세부터29세생활인구수  float64
 9   남자30세부터34세생활인구수  float64
 10  남자35세부터39세생활인구수  float64
 11  남자40세부터44세생활인구수  float64
 12  남자45세부터49세생활인구수  float64
 13  남자50세부터54세생활인구수  float64
 14  남자55세부터59세생활인구수  float64
 15  남자60세부터64세생활인구수  float64
 16  남자65세부터69세생활인구수  float64
 17  남자70세이상생활인구수     float64
 18  여자0세부터9세생활인구수    float64
 19  여자10세부터14세생활인구수  float64
 20  여자15세부터19세생활인구수  float64
 21  여자20세부터24세생활인구수  float64
 22  여자25세부터29세생활인구수  float64
 23  여자30세부터34세생활인구수  float64
 24  여자35세부터39세생활인구수  float64
 25  여자40세부터44세생활인구수  float64
 26  여자45세부터49세생활인구수  float64
 27  여자50세부터54세생활

## 2020 생활인구

In [129]:
# 2020 생활인구
Local202001=pd.read_csv(r'LOCAL_PEOPLE_DONG_202001.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202001=Local202001.reset_index()
Local202001=Local202001.shift(periods=6,axis=1)
Local202001=Local202001.drop(columns=['level_0','level_1','level_2','level_3','level_4','level_5'])

Local202002=pd.read_csv(r'LOCAL_PEOPLE_DONG_202002.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202002=Local202002.reset_index()
Local202002=Local202002.shift(periods=1,axis=1)
Local202002=Local202002.drop(columns=['index'])

Local202003=pd.read_csv(r'LOCAL_PEOPLE_DONG_202003.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202003=Local202003.reset_index()
Local202003=Local202003.shift(periods=1,axis=1)
Local202003=Local202003.drop(columns=['index'])

Local202004=pd.read_csv(r'LOCAL_PEOPLE_DONG_202004.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202004=Local202004.reset_index()
Local202004=Local202004.shift(periods=1,axis=1)
Local202004=Local202004.drop(columns=['index'])

Local202005=pd.read_csv(r'LOCAL_PEOPLE_DONG_202005.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202005=Local202005.reset_index()
Local202005=Local202005.shift(periods=1,axis=1)
Local202005=Local202005.drop(columns=['index'])

Local202006=pd.read_csv(r'LOCAL_PEOPLE_DONG_202006.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202006=Local202006.reset_index()
Local202006=Local202006.shift(periods=1,axis=1)
Local202006=Local202006.drop(columns=['index'])

Local202007=pd.read_csv(r'LOCAL_PEOPLE_DONG_202007.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202007=Local202007.reset_index()
Local202007=Local202007.shift(periods=1,axis=1)
Local202007=Local202007.drop(columns=['index'])

Local202008=pd.read_csv(r'LOCAL_PEOPLE_DONG_202008.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202008=Local202008.reset_index()
Local202008=Local202008.shift(periods=1,axis=1)
Local202008=Local202008.drop(columns=['index'])

Local202009=pd.read_csv(r'LOCAL_PEOPLE_DONG_202009.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202009=Local202009.reset_index()
Local202009=Local202009.shift(periods=1,axis=1)
Local202009=Local202009.drop(columns=['index'])

Local202010=pd.read_csv(r'LOCAL_PEOPLE_DONG_202010.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202010=Local202010.reset_index()
Local202010=Local202010.shift(periods=1,axis=1)
Local202010=Local202010.drop(columns=['index'])

Local202011=pd.read_csv(r'LOCAL_PEOPLE_DONG_202011.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202011=Local202011.reset_index()
Local202011=Local202011.shift(periods=1,axis=1)
Local202011=Local202011.drop(columns=['index'])

Local202012=pd.read_csv(r'LOCAL_PEOPLE_DONG_202012.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202012=Local202012.reset_index()
Local202012=Local202012.shift(periods=1,axis=1)
Local202012=Local202012.drop(columns=['index'])

Local2020 = pd.concat([Local202003,Local202004,Local202005,Local202006,Local202007,Local202008,Local202009,
                      Local202010,Local202011,Local202012], ignore_index = True)

##Local2020 = Local2020[['기준일ID','시간대구분','행정동코드','총생활인구수']]
Local2020.행정동코드 = Local2020.행정동코드 * 100
Local2020.총생활인구수 = Local2020.총생활인구수 * 10000
Local2020 = Local2020.astype({'총생활인구수':'int64'})

## 2021 생활인구

In [130]:
# 2021 생활인구
Local202101=pd.read_csv(r'LOCAL_PEOPLE_DONG_202101.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202101=Local202101.reset_index()
Local202101=Local202101.shift(periods=1,axis=1)
Local202101=Local202101.drop(columns=['index'])

Local202102=pd.read_csv(r'LOCAL_PEOPLE_DONG_202102.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202102=Local202102.reset_index()
Local202102=Local202102.shift(periods=1,axis=1)
Local202102=Local202102.drop(columns=['index'])

Local202103=pd.read_csv(r'LOCAL_PEOPLE_DONG_202103.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202103=Local202103.reset_index()
Local202103=Local202103.shift(periods=1,axis=1)
Local202103=Local202103.drop(columns=['index'])

Local202104=pd.read_csv(r'LOCAL_PEOPLE_DONG_202104.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202104=Local202104.reset_index()
Local202104=Local202104.shift(periods=1,axis=1)
Local202104=Local202104.drop(columns=['index'])

Local202105=pd.read_csv(r'LOCAL_PEOPLE_DONG_202105.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202105=Local202105.reset_index()
Local202105=Local202105.shift(periods=1,axis=1)
Local202105=Local202105.drop(columns=['index'])

Local202106=pd.read_csv(r'LOCAL_PEOPLE_DONG_202106.csv', engine='python', error_bad_lines=False, encoding='UTF8')
Local202106=Local202106.reset_index()
Local202106=Local202106.shift(periods=1,axis=1)
Local202106=Local202106.drop(columns=['index'])

Local2021 = pd.concat([Local202103,Local202104,Local202105,Local202106], ignore_index = True)

##Local2021 = Local2021[['기준일ID','시간대구분','행정동코드','총생활인구수']]
Local2021.행정동코드 = Local2021.행정동코드 * 100
Local2021.총생활인구수 = Local2021.총생활인구수 * 10000
Local2021 = Local2021.astype({'총생활인구수':'int64'})



In [131]:
BeforeCoronaLocal = Local2019
AfterCoronaLocal = pd.concat([Local2020,Local2021],ignore_index = True)
TotalLocal  = pd.concat([AfterCoronaLocal,BeforeCoronaLocal],ignore_index = True)

#### ** 코로나 전후 내국인 : TotalLoca(2019-2021), BeforeCoronaLocal(2019), AfterCoronaLoca(2020-2021)

In [132]:
TotalLocal['생활지역권'] = TotalLocal['행정동코드'].map(regionfunction)

In [133]:
BeforeCoronaLocal['생활지역권'] = BeforeCoronaLocal['행정동코드'].map(regionfunction)
AfterCoronaLocal['생활지역권'] = AfterCoronaLocal['행정동코드'].map(regionfunction)

# 2. 가이드 제공(지역생활권, 영업시간, 업종에 따라)

In [134]:
BeforeCoronaLocalcopy=copy.deepcopy(BeforeCoronaLocal)

BeforeCoronaLocalcopy['성별']='남자'
import pandas as pd

BeforeCoronaLocalMan=pd.melt(BeforeCoronaLocalcopy, id_vars=['생활지역권', '시간대구분','성별'], value_vars=['남자0세부터9세생활인구수', '남자10세부터14세생활인구수', '남자15세부터19세생활인구수','남자20세부터24세생활인구수', '남자25세부터29세생활인구수', '남자30세부터34세생활인구수', '남자35세부터39세생활인구수',
    '남자40세부터44세생활인구수', '남자45세부터49세생활인구수', '남자50세부터54세생활인구수','남자55세부터59세생활인구수', 
    '남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '남자70세이상생활인구수'])
BeforeCoronaLocalcopy.drop(['성별'], axis='columns')

BeforeCoronaLocalcopy['성별']='여자'
BeforeCoronaLocalWoman=pd.melt(BeforeCoronaLocalcopy, id_vars=['생활지역권', '시간대구분','성별'], value_vars=['여자0세부터9세생활인구수', '여자10세부터14세생활인구수', '여자15세부터19세생활인구수','여자20세부터24세생활인구수', '여자25세부터29세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수',
    '여자40세부터44세생활인구수', '여자45세부터49세생활인구수', '여자50세부터54세생활인구수','여자55세부터59세생활인구수', 
    '여자60세부터64세생활인구수', '여자65세부터69세생활인구수', '여자70세이상생활인구수'])
AfterCoronaLocalcopy=copy.deepcopy(AfterCoronaLocal)

AfterCoronaLocalcopy['성별']='남자'
import pandas as pd
AfterCoronaLocalMan=pd.melt(AfterCoronaLocalcopy, id_vars=['생활지역권', '시간대구분','성별'], value_vars=['남자0세부터9세생활인구수', '남자10세부터14세생활인구수', '남자15세부터19세생활인구수','남자20세부터24세생활인구수', '남자25세부터29세생활인구수', '남자30세부터34세생활인구수', '남자35세부터39세생활인구수',
    '남자40세부터44세생활인구수', '남자45세부터49세생활인구수', '남자50세부터54세생활인구수','남자55세부터59세생활인구수', 
    '남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '남자70세이상생활인구수'])
AfterCoronaLocalcopy.drop(['성별'], axis='columns')

AfterCoronaLocalcopy['성별']='여자'
AfterCoronaLocalWoman=pd.melt(AfterCoronaLocalcopy, id_vars=['생활지역권', '시간대구분','성별'], value_vars=['여자0세부터9세생활인구수', '여자10세부터14세생활인구수', '여자15세부터19세생활인구수','여자20세부터24세생활인구수', '여자25세부터29세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수',
    '여자40세부터44세생활인구수', '여자45세부터49세생활인구수', '여자50세부터54세생활인구수','여자55세부터59세생활인구수', 
    '여자60세부터64세생활인구수', '여자65세부터69세생활인구수', '여자70세이상생활인구수'])

In [135]:
def sexagefunction(x):
    if x in ['남자0세부터9세생활인구수', '남자10세부터14세생활인구수', '남자15세부터19세생활인구수']:
        return "남자 20대 이하";
    if x in ['남자20세부터24세생활인구수', '남자25세부터29세생활인구수']:
        return "남자 20대";
    if x in ['남자30세부터34세생활인구수', '남자35세부터39세생활인구수']:
        return "남자 30대";
    if x in ['남자40세부터44세생활인구수', '남자45세부터49세생활인구수']:
        return "남자 40대"
    if x in ['남자50세부터54세생활인구수','남자55세부터59세생활인구수']:
        return "남자 50대"
    if x in ['남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '남자70세이상생활인구수']:
        return '남자 60대 이상'
    if x in ['여자0세부터9세생활인구수', '여자10세부터14세생활인구수', '여자15세부터19세생활인구수']:
        return "여자 20대 이하"
    if x in ['여자20세부터24세생활인구수', '여자25세부터29세생활인구수']:
        return "여자 20대"
    if x in ['여자30세부터34세생활인구수', '여자35세부터39세생활인구수']:
        return "여자 30대"
    if x in ['여자40세부터44세생활인구수', '여자45세부터49세생활인구수']:
        return "여자 40대"
    if x in ['여자50세부터54세생활인구수','여자55세부터59세생활인구수']:
        return "여자 50대"
    if x in ['여자60세부터64세생활인구수', '여자65세부터69세생활인구수', '여자70세이상생활인구수']:
        return '여자 60대 이상'

In [136]:
BeforeCoronaLocalMan['연령대']=BeforeCoronaLocalMan['variable'].map(sexagefunction)
BeforeCoronaLocalWoman['연령대']=BeforeCoronaLocalWoman['variable'].map(sexagefunction)
BeforeCoronaLocalMan.rename(columns = {'value' : '생활인구수'}, inplace = True)
BeforeCoronaLocalWoman.rename(columns = {'value' : '생활인구수'}, inplace = True)
AfterCoronaLocalMan['연령대']=AfterCoronaLocalMan['variable'].map(sexagefunction)
AfterCoronaLocalWoman['연령대']=AfterCoronaLocalWoman['variable'].map(sexagefunction)
AfterCoronaLocalMan.rename(columns = {'value' : '생활인구수'}, inplace = True)
AfterCoronaLocalWoman.rename(columns = {'value' : '생활인구수'}, inplace = True)

In [137]:
BeforeCoronaLocalManSex=BeforeCoronaLocalMan.groupby(by=['생활지역권','시간대구분']).sum()[['생활인구수']]
BeforeCoronaLocalWomanSex=BeforeCoronaLocalWoman.groupby(by=['생활지역권','시간대구분']).sum()[['생활인구수']]
BeforeCoronaLocalManSex.reset_index(inplace=True)
BeforeCoronaLocalWomanSex.reset_index(inplace=True)

BeforeCoronaLocalManAge=BeforeCoronaLocalMan.groupby(by=['생활지역권','시간대구분', '연령대']).sum()[['생활인구수']]
BeforeCoronaLocalWomanAge=BeforeCoronaLocalWoman.groupby(by=['생활지역권','시간대구분','연령대']).sum()[['생활인구수']]
BeforeCoronaLocalManAge.reset_index(inplace=True)
BeforeCoronaLocalWomanAge.reset_index(inplace=True)


AfterCoronaLocalManSex=AfterCoronaLocalMan.groupby(by=['생활지역권','시간대구분']).sum()[['생활인구수']]
AfterCoronaLocalWomanSex=AfterCoronaLocalWoman.groupby(by=['생활지역권','시간대구분']).sum()[['생활인구수']]
AfterCoronaLocalManSex.reset_index(inplace=True)
AfterCoronaLocalWomanSex.reset_index(inplace=True)

AfterCoronaLocalManAge=AfterCoronaLocalMan.groupby(by=['생활지역권','시간대구분', '연령대']).sum()[['생활인구수']]
AfterCoronaLocalWomanAge=AfterCoronaLocalWoman.groupby(by=['생활지역권','시간대구분','연령대']).sum()[['생활인구수']]
AfterCoronaLocalManAge.reset_index(inplace=True)
AfterCoronaLocalWomanAge.reset_index(inplace=True)

In [149]:
## 시간을 영업시간으로 변환하는 함수
def timefunction(x):
    if x in [0, 1, 2, 3, 4, 5]:
        return '새벽'
    if x in [6, 7, 8, 9, 10, 11]:
        return '아침'
    if x in [12, 13, 14, 15, 16, 17]:
        return '점심'
    if x in [18, 19, 20, 21, 22, 23]:
        return '저녁'

## - 지역 생활권 상권분석, 
## - (코로나 전<->후) 특정 생활지역권  성별 연령별 구매력, 전체 업종별 선호도(매출건수),업종의 성별 연령별 업종별 선호도
## ipywidgets - 버튼구현(선택한 버튼에 따라 결과가 다르게 표현)
## plotly 이용해서 interactive_graph 구현(그래프 위에 마우스를 올리면 관련 정보를 얻을 수 있고 , 범례를 클릭하면 특정 그래프의 요소를 제거 가능)

In [150]:
##버튼 내용 생성
import pandas as pd
import numpy as np
import ipywidgets as widgets
from plotly.subplots import make_subplots
import plotly.graph_objects as go
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique
selectregion=unique_sorted_values_plus_ALL(TotalLocal["생활지역권"])
selectregion[0]="지역생활권을 선택하세요"
#selectregion

selectmenu=["업종을 선택하세요", "한식", '일식/수산물', '닭/오리요리', '양식', '분식']

In [140]:
from IPython.display import clear_output
import ipywidgets as widgets
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
##지역 생활권 dropdown
dropdown_region = widgets.Dropdown(
    description='지역생활권',options=selectregion)
##영업시간 위젯
time_buttons = widgets.ToggleButtons(
    options=[('전체', '0-24'),('아침', '6-12'), ('점심', '13-18'), ('저녁', '18-24'),('새벽', '0-6')],
    description='영업시간별',
)
#업종 위젯
dropdown_menu = widgets.Dropdown(
    description='업종별',
    options=selectmenu)
# 위젯을 container로 합침.
container = widgets.VBox([
                widgets.HBox([dropdown_region]),
                widgets.HBox([time_buttons]),
                widgets.HBox([dropdown_menu]),
                ])


def response(change):
    # 위젯에 변화가 감지되면 실행되는 코드
    # 이전 셀 아웃풋은 삭제
    clear_output()  
    # 위젯 출력
    print('******************************************************************\n')
    print('지역권생활권별 요식업종 가이드\n')
    print('1. 지역생활권 이란?')
    print('서울시 지역을 생활권 단위로 나누어 도시관리 계획을 구체적으로 제시하기 위한 공간 관리 지침\n')
    display(container)
    print('2. '+dropdown_region.value+"지역 생활권 상권분석")
    istemp=whichregion['생활지역권']==dropdown_region.value
    tempregion=whichregion[istemp]
    tempregion=tempregion['행정동명'].unique()
    print("행정구역 : "+','.join(tempregion))

    #print(tempregion)
    region=make_subplots(rows=1, cols=1)
    #region = px.data.tips()
    tempRegion=df2[df2['생활지역권']==dropdown_region.value]
    tempRegion=tempRegion.groupby(by=['중분류명','소분류명']).count()[['생활지역권']]
    tempRegion=tempRegion.reset_index()
    tempRegion['상가수']=tempRegion['생활지역권']
    region = px.bar(tempRegion, x="상가수", y="중분류명", color='소분류명', orientation='h',
             height=400,
             title='<b>2-1 '+dropdown_region.value+'지역 생활권 업종별 상가 분석</b>')
    
    display(region)
    print("업종위에 마우스를 올리시면" +dropdown_region.value+" 지역 생활권의 정확한 상가 개수를 알 수 있습니다.")
   
    
    consumersbefore=make_subplots(rows=1, cols=2)
    consumersbefore.update_layout(title='<b>2-2-1(코로나 전) '+dropdown_region.value+ ' 생활지역권 시간에 따른 성별 연령별 생활인구</b>')
    tempMan=BeforeCoronaLocalManSex[BeforeCoronaLocalManSex["생활지역권"]==dropdown_region.value]
    tempWoman=BeforeCoronaLocalWomanSex[BeforeCoronaLocalWomanSex["생활지역권"]==dropdown_region.value]
    consumersbefore.add_trace(go.Scatter(x=tempMan['시간대구분'],y=tempMan['생활인구수'], mode='lines+markers', name='남자'), row=1, col=1)
    consumersbefore.add_trace(go.Scatter(x=tempWoman['시간대구분'],y=tempWoman['생활인구수'], mode='lines+markers', name='여자'), row=1, col=1)
    consumersbefore.add_trace(go.Scatter(x=tempWoman['시간대구분'],y=tempMan['생활인구수']+tempWoman['생활인구수'], mode='lines+markers', name='내국인'), row=1, col=1)
    
    isDongAge=(BeforeCoronaLocalManAge['생활지역권']==dropdown_region.value)&(BeforeCoronaLocalManAge['연령대']== "남자 20대 이하")
    tempMan20U=BeforeCoronaLocalManAge[isDongAge]
    isDongAge=(BeforeCoronaLocalManAge['생활지역권']==dropdown_region.value)&(BeforeCoronaLocalManAge['연령대']== "남자 20대")
    tempMan20=BeforeCoronaLocalManAge[isDongAge]
    isDongAge=(BeforeCoronaLocalManAge['생활지역권']==dropdown_region.value)&(BeforeCoronaLocalManAge['연령대']== "남자 30대")
    tempMan30=BeforeCoronaLocalManAge[isDongAge]
    isDongAge=(BeforeCoronaLocalManAge['생활지역권']==dropdown_region.value)&(BeforeCoronaLocalManAge['연령대']== "남자 40대")
    tempMan40=BeforeCoronaLocalManAge[isDongAge]
    isDongAge=(BeforeCoronaLocalManAge['생활지역권']==dropdown_region.value)&(BeforeCoronaLocalManAge['연령대']== "남자 50대")
    tempMan50=BeforeCoronaLocalManAge[isDongAge]
    isDongAge=(BeforeCoronaLocalManAge['생활지역권']==dropdown_region.value)&(BeforeCoronaLocalManAge['연령대']== "남자 60대 이상")
    tempMan60U=BeforeCoronaLocalManAge[isDongAge]

    isDongAge=(BeforeCoronaLocalWomanAge['생활지역권']==dropdown_region.value)&(BeforeCoronaLocalWomanAge['연령대']== "여자 20대 이하")
    tempWoman20U=BeforeCoronaLocalWomanAge[isDongAge]
    isDongAge=(BeforeCoronaLocalWomanAge['생활지역권']==dropdown_region.value)&(BeforeCoronaLocalWomanAge['연령대']== "여자 20대")
    tempWoman20=BeforeCoronaLocalWomanAge[isDongAge]
    isDongAge=(BeforeCoronaLocalWomanAge['생활지역권']==dropdown_region.value)&(BeforeCoronaLocalWomanAge['연령대']== "여자 30대")
    tempWoman30=BeforeCoronaLocalWomanAge[isDongAge]
    isDongAge=(BeforeCoronaLocalWomanAge['생활지역권']==dropdown_region.value)&(BeforeCoronaLocalWomanAge['연령대']== "여자 40대")
    tempWoman40=BeforeCoronaLocalWomanAge[isDongAge]
    isDongAge=(BeforeCoronaLocalWomanAge['생활지역권']==dropdown_region.value)&(BeforeCoronaLocalWomanAge['연령대']== "여자 50대")
    tempWoman50=BeforeCoronaLocalWomanAge[isDongAge]
    isDongAge=(BeforeCoronaLocalWomanAge['생활지역권']==dropdown_region.value)&(BeforeCoronaLocalWomanAge['연령대']== "여자 60대 이상")
    tempWoman60U=BeforeCoronaLocalWomanAge[isDongAge]

    consumersbefore.add_trace(go.Scatter(x=tempMan20U['시간대구분'],y=tempMan20U['생활인구수']+tempWoman20U['생활인구수'], mode='lines+markers', name='20 Under'), row=1, col=2)
    consumersbefore.add_trace(go.Scatter(x=tempMan20['시간대구분'],y=tempMan20['생활인구수']+tempWoman20['생활인구수'], mode='lines+markers', name='20s'), row=1, col=2)
    consumersbefore.add_trace(go.Scatter(x=tempMan30['시간대구분'],y=tempMan30['생활인구수']+tempWoman30['생활인구수'], mode='lines+markers', name='30s'), row=1, col=2)
    consumersbefore.add_trace(go.Scatter(x=tempMan40['시간대구분'],y=tempMan40['생활인구수']+tempWoman40['생활인구수'], mode='lines+markers', name='40s'), row=1, col=2)
    consumersbefore.add_trace(go.Scatter(x=tempMan50['시간대구분'],y=tempMan50['생활인구수']+tempWoman50['생활인구수'], mode='lines+markers', name='50s'), row=1, col=2)
    consumersbefore.add_trace(go.Scatter(x=tempMan60U['시간대구분'],y=tempMan60U['생활인구수']+tempWoman60U['생활인구수'], mode='lines+markers', name='60 Upper'), row=1, col=2)
    display(consumersbefore)
    
    ##################################################################################################################
    consumersafter=make_subplots(rows=1, cols=2)
    consumersafter.update_layout(title='<b>2-2-2(코로나 후) '+dropdown_region.value+ ' 생활지역권 시간에 따른 성별 연령별 생활인구</b>')
    tempMan=AfterCoronaLocalManSex[AfterCoronaLocalManSex["생활지역권"]==dropdown_region.value]
    tempWoman=AfterCoronaLocalWomanSex[AfterCoronaLocalWomanSex["생활지역권"]==dropdown_region.value]

    consumersafter.add_trace(go.Scatter(x=tempMan['시간대구분'],y=tempMan['생활인구수'], mode='lines+markers', name='남자'), row=1, col=1)
    consumersafter.add_trace(go.Scatter(x=tempWoman['시간대구분'],y=tempWoman['생활인구수'], mode='lines+markers', name='여자'), row=1, col=1)
    consumersafter.add_trace(go.Scatter(x=tempWoman['시간대구분'],y=tempMan['생활인구수']+tempWoman['생활인구수'], mode='lines+markers', name='내국인'), row=1, col=1)
    
    isDongAge=(AfterCoronaLocalManAge['생활지역권']==dropdown_region.value)&(AfterCoronaLocalManAge['연령대']== "남자 20대 이하")
    tempMan20U=AfterCoronaLocalManAge[isDongAge]
    isDongAge=(AfterCoronaLocalManAge['생활지역권']==dropdown_region.value)&(AfterCoronaLocalManAge['연령대']== "남자 20대")
    tempMan20=AfterCoronaLocalManAge[isDongAge]
    isDongAge=(AfterCoronaLocalManAge['생활지역권']==dropdown_region.value)&(AfterCoronaLocalManAge['연령대']== "남자 30대")
    tempMan30=AfterCoronaLocalManAge[isDongAge]
    isDongAge=(AfterCoronaLocalManAge['생활지역권']==dropdown_region.value)&(AfterCoronaLocalManAge['연령대']== "남자 40대")
    tempMan40=AfterCoronaLocalManAge[isDongAge]
    isDongAge=(AfterCoronaLocalManAge['생활지역권']==dropdown_region.value)&(AfterCoronaLocalManAge['연령대']== "남자 50대")
    tempMan50=AfterCoronaLocalManAge[isDongAge]
    isDongAge=(AfterCoronaLocalManAge['생활지역권']==dropdown_region.value)&(AfterCoronaLocalManAge['연령대']== "남자 60대 이상")
    tempMan60U=AfterCoronaLocalManAge[isDongAge]

    isDongAge=(AfterCoronaLocalWomanAge['생활지역권']==dropdown_region.value)&(AfterCoronaLocalWomanAge['연령대']== "여자 20대 이하")
    tempWoman20U=AfterCoronaLocalWomanAge[isDongAge]
    isDongAge=(AfterCoronaLocalWomanAge['생활지역권']==dropdown_region.value)&(AfterCoronaLocalWomanAge['연령대']== "여자 20대")
    tempWoman20=AfterCoronaLocalWomanAge[isDongAge]
    isDongAge=(AfterCoronaLocalWomanAge['생활지역권']==dropdown_region.value)&(AfterCoronaLocalWomanAge['연령대']== "여자 30대")
    tempWoman30=AfterCoronaLocalWomanAge[isDongAge]
    isDongAge=(AfterCoronaLocalWomanAge['생활지역권']==dropdown_region.value)&(AfterCoronaLocalWomanAge['연령대']== "여자 40대")
    tempWoman40=AfterCoronaLocalWomanAge[isDongAge]
    isDongAge=(AfterCoronaLocalWomanAge['생활지역권']==dropdown_region.value)&(AfterCoronaLocalWomanAge['연령대']== "여자 50대")
    tempWoman50=AfterCoronaLocalWomanAge[isDongAge]
    isDongAge=(AfterCoronaLocalWomanAge['생활지역권']==dropdown_region.value)&(AfterCoronaLocalWomanAge['연령대']== "여자 60대 이상")
    tempWoman60U=AfterCoronaLocalWomanAge[isDongAge]

    consumersafter.add_trace(go.Scatter(x=tempMan20U['시간대구분'],y=tempMan20U['생활인구수']+tempWoman20U['생활인구수'], mode='lines+markers', name='20 Under'), row=1, col=2)
    consumersafter.add_trace(go.Scatter(x=tempMan20['시간대구분'],y=tempMan20['생활인구수']+tempWoman20['생활인구수'], mode='lines+markers', name='20s'), row=1, col=2)
    consumersafter.add_trace(go.Scatter(x=tempMan30['시간대구분'],y=tempMan30['생활인구수']+tempWoman30['생활인구수'], mode='lines+markers', name='30s'), row=1, col=2)
    consumersafter.add_trace(go.Scatter(x=tempMan40['시간대구분'],y=tempMan40['생활인구수']+tempWoman40['생활인구수'], mode='lines+markers', name='40s'), row=1, col=2)
    consumersafter.add_trace(go.Scatter(x=tempMan50['시간대구분'],y=tempMan50['생활인구수']+tempWoman50['생활인구수'], mode='lines+markers', name='50s'), row=1, col=2)
    consumersafter.add_trace(go.Scatter(x=tempMan60U['시간대구분'],y=tempMan60U['생활인구수']+tempWoman60U['생활인구수'], mode='lines+markers', name='60 Upper'), row=1, col=2)

    display(consumersafter)
    
############################################################################################
    consumersrate = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])
    BeforeCoronaLocalManWoman=copy.deepcopy(BeforeCoronaLocalManAge)
    BeforeCoronaLocalManWoman=BeforeCoronaLocalManWoman.append(BeforeCoronaLocalWomanAge)
    BeforeCoronaLocalManWoman=BeforeCoronaLocalManWoman[BeforeCoronaLocalManWoman['생활지역권']==dropdown_region.value]
    BeforeCoronaLocalManWoman['영업시간']=BeforeCoronaLocalManWoman['시간대구분'].map(timefunction)
    BeforeCoronaLocalManWoman.drop('시간대구분', axis=1, inplace=True)
    BeforeCoronaLocalManWoman=BeforeCoronaLocalManWoman.groupby(['연령대', '영업시간']).sum(['생활인구수'])
    BeforeCoronaLocalManWoman.reset_index(inplace=True)
    if time_buttons.label!='전체':
        BeforeCoronaLocalManWoman=BeforeCoronaLocalManWoman[BeforeCoronaLocalManWoman['영업시간']==time_buttons.label]
    consumersrate.add_trace(go.Pie(labels=BeforeCoronaLocalManWoman['연령대'].unique().tolist(), values=BeforeCoronaLocalManWoman['생활인구수'], hole=.3, name="코로나 전 생활인구"), row=1, col=1)

    AfterCoronaLocalManWoman=copy.deepcopy(AfterCoronaLocalManAge)
    AfterCoronaLocalManWoman=AfterCoronaLocalManWoman.append(AfterCoronaLocalWomanAge)
    AfterCoronaLocalManWoman=AfterCoronaLocalManWoman[AfterCoronaLocalManWoman['생활지역권']==dropdown_region.value]
    AfterCoronaLocalManWoman['영업시간']=AfterCoronaLocalManWoman['시간대구분'].map(timefunction)
    AfterCoronaLocalManWoman.drop('시간대구분', axis=1, inplace=True)
    AfterCoronaLocalManWoman=AfterCoronaLocalManWoman.groupby(['연령대', '영업시간']).sum(['생활인구수'])
    AfterCoronaLocalManWoman.reset_index(inplace=True)
    if time_buttons.label!='전체':
        AfterCoronaLocalManWoman=AfterCoronaLocalManWoman[AfterCoronaLocalManWoman['영업시간']==time_buttons.label]
    consumersrate.add_trace(go.Pie(labels=BeforeCoronaLocalManWoman['연령대'].unique().tolist(), values=AfterCoronaLocalManWoman['생활인구수'], hole=.3, name="코로나 후 생활인구"), row=1, col=2)
    consumersrate.update_layout(title="<b>2-2-3["+str(time_buttons.label)+"] 시간 ["+str(dropdown_region.value)+"] 생활 지역권 코로나 전후 가락 저녁 생활인구 비율 비교</b>")
    display(consumersrate)
###############################################################################################
    
    paypower=make_subplots(rows=1, cols=2, specs=[[{"type": "bar"}, {"type": "bar"}]])
    paypower.update_layout(title='<b>3-1(코로나 전<->후) '+dropdown_region.value+ ' 생활지역권  성별 연령별 구매력</b>')

    paypower.add_trace(go.Bar(x=dfb_group[dfb_group['연령대'] == '2.30대'].성별구분, y=dfb_group[dfb_group['연령대'] == '2.30대'].매출건수, name='2.30대'), row=1, col=1)
    paypower.add_trace(go.Bar(x=dfb_group[dfb_group['연령대'] == '3.40대'].성별구분, y=dfb_group[dfb_group['연령대'] == '3.40대'].매출건수, name='3.40대'), row=1, col=1)
    paypower.add_trace(go.Bar(x=dfb_group[dfb_group['연령대'] == '4.50대'].성별구분, y=dfb_group[dfb_group['연령대'] == '4.50대'].매출건수, name='4.50대'), row=1, col=1)
    paypower.add_trace(go.Bar(x=dfb_group[dfb_group['연령대'] == '5.60대'].성별구분, y=dfb_group[dfb_group['연령대'] == '5.60대'].매출건수, name='5.60대'), row=1, col=1)
    paypower.add_trace(go.Bar(x=dfb_group[dfb_group['연령대'] == '2.30대'].성별구분, y=dfa_group[dfa_group['연령대'] == '2.30대'].매출건수, name='2.30대'), row=1, col=2)
    paypower.add_trace(go.Bar(x=dfb_group[dfb_group['연령대'] == '3.40대'].성별구분, y=dfa_group[dfa_group['연령대'] == '3.40대'].매출건수, name='3.40대'), row=1, col=2)
    paypower.add_trace(go.Bar(x=dfb_group[dfb_group['연령대'] == '4.50대'].성별구분, y=dfa_group[dfa_group['연령대'] == '4.50대'].매출건수, name='4.50대'), row=1, col=2)
    paypower.add_trace(go.Bar(x=dfb_group[dfb_group['연령대'] == '5.60대'].성별구분, y=dfa_group[dfa_group['연령대'] == '5.60대'].매출건수, name='5.60대'), row=1, col=2)
    display(paypower)
    
###################################################################################################
    allmenu=go.Figure()
    allmenu.update_layout(title="<b>3-2 (코로나 전<->후) 전체 업종별 선호도(매출건수)</b>")
    def myfood(foodname):
        alldf=pie_food[pie_food['중분류명']==foodname]
        ##2019 : 10달 2020-2021 :15달 S=> eoul['날짜'].unique()결과
        alldf['코로나 전후']=alldf['기준년도'].map(isbefore)
        alldf=alldf.groupby(['코로나 전후']).sum(['매출건수'])
        alldf=alldf.reset_index()
        alldfxlist=["코로나 전", "코로나 후"]
        alldfylist=[alldf.loc[0]['매출건수'],alldf.loc[1]['매출건수'] ]
        alldfylist[0]/=10
        alldfylist[1]/=15
        return alldfxlist, alldfylist
    tempfood1=myfood('한식')
    allmenu.add_trace(go.Scatter(x=tempfood1[0],y=tempfood1[1], mode='lines+markers', name='한식'))
    tempfood2=myfood('일식/수산물')
    allmenu.add_trace(go.Scatter(x=tempfood2[0],y=tempfood2[1], mode='lines+markers', name='일식/수산물'))
    tempfood3=myfood('닭/오리요리')
    allmenu.add_trace(go.Scatter(x=tempfood3[0],y=tempfood3[1], mode='lines+markers', name='닭/오리고기'))
    tempfood4=myfood('양식')
    allmenu.add_trace(go.Scatter(x=tempfood4[0],y=tempfood4[1], mode='lines+markers', name='양식'))
    tempfood5=myfood('분식')
    allmenu.add_trace(go.Scatter(x=tempfood5[0],y=tempfood5[1], mode='lines+markers', name='분식'))
    display(allmenu)
    
#################################################################################3
    menu=make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])
    menu.update_layout(title="<b>3-3 ["+dropdown_menu.value+"] 업종 코로나 전후 성별 연령별 업종별 선호도</b>")
    pie_foodbefore=pie_food[pie_food['기준년도']=='2019']
    pie_foodbefore=pie_foodbefore[pie_foodbefore['중분류명']==dropdown_menu.value]
    pie_foodbefore=pie_foodbefore.groupby(['new성별연령', '중분류명']).sum(['매출건수'])
    pie_foodbefore = pie_foodbefore.reset_index()
    pie_foodafter=pie_food[pie_food['기준년도']!="2019"]
    pie_foodafter=pie_foodafter[pie_foodafter['중분류명']==dropdown_menu.value]
    pie_foodafter=pie_foodafter.groupby(['new성별연령', '중분류명']).sum(['매출건수'])
    pie_foodafter = pie_foodafter.reset_index()
    if(dropdown_menu.value=="업종을 선택하세요"):
        print('<b>3-3 [선택] 업종 코로나 전후 성별 연령별 업종별 선호도</b>')
        print("업종을 선택하시면 특정 업종의 성별 연령별 선호도를 볼 수 있습니다")
    else:
        menu.add_trace(go.Pie(labels=pie_foodbefore['new성별연령'].unique().tolist(), values=pie_foodbefore['매출건수'], hole=.3, name="코로나 전 매출건수 비율"), row=1, col=1)
        menu.add_trace(go.Pie(labels=pie_foodafter['new성별연령'].unique().tolist(), values=pie_foodafter['매출건수'], hole=.3, name="코로나 전 매출건수 비율"), row=1, col=2)
        display(menu)
    
# 위젯 변화를 observe하여 response 함수를 실행
dropdown_region.observe(response, names='value')
time_buttons.observe(response, names="value")
dropdown_menu.observe(response, names="value")
print('******************************************************************\n')
print('지역권생활권별 요식업종 가이드\n')
print('1. 지역생활권 이란?')
print('서울시 지역을 생활권 단위로 나누어 도시관리 계획을 구체적으로 제시하기 위한 공간 관리 지침\n')
print('**분석하고 싶은 지역생활권을 선택하세요')
display(container)

******************************************************************

지역권생활권별 요식업종 가이드

1. 지역생활권 이란?
서울시 지역을 생활권 단위로 나누어 도시관리 계획을 구체적으로 제시하기 위한 공간 관리 지침

**분석하고 싶은 지역생활권을 선택하세요


## 2-4 지역생활권별, 영업시간<->업종별 추천
### [성. 연령별 업종별 구매건수 : 성. 연령별:12개  업종5개)
#### arraynumpy1 : 연령별 중분류명 매출건수 (닭/오리요리, 분식, 양식, 일식/수산물, 한식)
#### arraynumpy2 : 지역생활권별 중분류명 상가수
#### arraynumpy3all (dawn, breakfast, afternoon, dinner) : 시계열 결과 만들어진 DF  (지역생활권별 예비소비자 예측 - 
#### https://github.com/CUAI-CAU/CUAI-FINANCE-TEAM2 
#### 시계열분석_전체, 시계열분석_새벽, 시계열 분석_아침, 시계열분석_점심, 시계열분석_저녁) 실행 결과 
#### finaldf.csv, dawn_data.csv, breakfast_data.csv, afternoon_data.csv, dinner_data.csv

In [142]:
arraydata1=pie_food[pie_food['기준년도']!="2019"]
arraydata1=arraydata1.groupby(by=['new성별연령','중분류명']).sum()[['매출건수']]
arraydata1=arraydata1.reset_index()
arraydata1=arraydata1.set_index('new성별연령')
arraydata1=arraydata1.reset_index()
arraydata11=arraydata1['매출건수'].to_numpy()
arraynumpy1=arraydata11.reshape((-1, 5))

arraydata2=copy.deepcopy(df2)
arraydata2=arraydata2.groupby(by=['생활지역권','중분류명']).count()
arraydata2=arraydata2.reset_index()
arraydata22=arraydata2[(arraydata2['중분류명']=='닭/오리요리')|(arraydata2['중분류명']=='분식')|(arraydata2['중분류명']=='양식')|(arraydata2['중분류명']=='일식/수산물')|(arraydata2['중분류명']=='한식')]
arraydata22=arraydata22['소분류명'].to_numpy()
arraynumpy2=arraydata22.reshape((-1, 5))

arraydata3all=pd.read_csv('finaldf.csv')
arraydata3dawn=pd.read_csv('dawn_data.csv')
arraydata3breakfast=pd.read_csv('breakfast_data.csv')
arraydata3afternoon=pd.read_csv('afternoon_data.csv')
arraydata3dinner=pd.read_csv('dinner_data.csv')

import numpy
region_name=arraydata2['생활지역권'].unique()
region_name_List=region_name.tolist()
mul=arraynumpy3.dot(arraynumpy1)
muldiv=mul/arraynumpy2
region_index=region_name_List.index("가산")
muldiv[region_index].argsort()
menulist=['닭/오리요리', "분식", '양식', '일식/수산물', '한식']


dropdown_region2 = widgets.Dropdown(
    description='지역생활권',options=selectregion)
##영업시간 위젯
time_buttons2 = widgets.ToggleButtons(
    options=[('전체', '0-24'),('아침', '6-12'), ('점심', '13-18'), ('저녁', '18-24'),('새벽', '0-6')],
    description='영업시간별',
)
#업종 위젯
dropdown_menu2 = widgets.Dropdown(
    description='업종별',
    options=selectmenu)
# 위젯을 container로 합침.
container1=widgets.VBox([widgets.HBox([dropdown_region2])])
container2 = widgets.VBox([widgets.HBox([time_buttons2])])
container3 = widgets.VBox([widgets.HBox([dropdown_menu2])])

def response2(change):
    # 위젯에 변화가 감지되면 실행되는 코드
    # 이전 셀 아웃풋은 삭제
    clear_output()  
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    print("\n4. 요식업 업종&영업시간 추천")
    display(container1)
    display(container2)
    print("\n["+dropdown_region2.value+"] 지역생활권 ["+time_buttons2.label+"] 영업시간 업종추천 : \n")
    import numpy
    region_name=arraydata2['생활지역권'].unique()
    region_name_List=region_name.tolist()
    ## 성 연령 : 20대 이하 남성, 30대 남성.... 60대, 20대 이사 여성, ... 60대 이상 여성 : 10개
    ## 업종(중분류명) : 닭/오리요리, 분식, 양식, 일식/수산물, 한식
    ## 지역생활권 : 125개

    #### arraynumpy1 : 성. 연령별 중분류명 매출건수 [10*5]
    #### arraynumpy2 : 지역생활권별 중분류명 상가수 [125*5]
    ### arraynumpy3 : 지역생활권별 성.연령별 예비소비자 [125*10]
    
    ##arraynumpy3*arraynumpy1/arraynumpy2[(125*10)X(10*5)/(125*5)] 
    ##내적곱 수행결과 지역생활권별 중분류 매출건수 예측
    ##나눗셈 수행 결과=지역생활권별 중분류 개업시 얻을 수 있는 매출건수
    
    arraydata3A=arraydata3all[arraydata3all['생활지역권']!='목동1']
    arraydata33A=arraydata3A.drop(columns='생활지역권')
    arraynumpy3A=arraydata33A.to_numpy()
    mulA=arraynumpy3A.dot(arraynumpy1)
    muldivA=mulA/arraynumpy2
    
    arraydata3Dawn=arraydata3dawn[arraydata3dawn['생활지역권']!='목동1']
    arraydata33Dawn=arraydata3Dawn.drop(columns='생활지역권')
    arraynumpy3Dawn=arraydata33Dawn.to_numpy()
    mulDawn=arraynumpy3Dawn.dot(arraynumpy1)
    muldivDawn=mulDawn/arraynumpy2
    
    arraydata3B=arraydata3breakfast[arraydata3breakfast['생활지역권']!='목동1']
    arraydata33B=arraydata3B.drop(columns='생활지역권')
    arraynumpy3B=arraydata33B.to_numpy()
    mulB=arraynumpy3B.dot(arraynumpy1)
    muldivB=mulB/arraynumpy2
    
    arraydata3L=arraydata3afternoon[arraydata3afternoon['생활지역권']!='목동1']
    arraydata33L=arraydata3L.drop(columns='생활지역권')
    arraynumpy3L=arraydata33L.to_numpy()
    mulL=arraynumpy3L.dot(arraynumpy1)
    muldivL=mulL/arraynumpy2
    
    arraydata3D=arraydata3dinner[arraydata3dinner['생활지역권']!='목동1']
    arraydata33D=arraydata3D.drop(columns='생활지역권')
    arraynumpy3D=arraydata33D.to_numpy()
    mulD=arraynumpy3D.dot(arraynumpy1)
    muldivD=mulD/arraynumpy2
    
    if time_buttons2.label=="전체":
        selectRegionTime=muldivA
    if time_buttons2.label=="새벽":
        selectRegionTime=muldivDawn
    if time_buttons2.label=="아침":
        selectRegionTime=muldivB
    if time_buttons2.label=="점심":
        selectRegionTime=muldivL
    if time_buttons2.label=="저녁":
        selectRegionTime=muldivD

    region_index=region_name_List.index(str(dropdown_region2.value))
    templist=selectRegionTime[region_index].argsort()
    nummenu=len(menulist)
    while nummenu!=0:
        print(str(5-nummenu+1)+". "+str(menulist[templist[nummenu-1]])+"\n")
        nummenu-=1
    
#################################################################################################################
    display(container3)
    if dropdown_menu2.value=="업종을 선택하세요":
        print("업종을 선택하시면 영업시간을 추천 받으실 수 있습니다.")
    else:
        print("["+dropdown_region2.value+"] 지역생활권 ["+dropdown_menu2.value+"] 업종 영업시간 추천 : ")
        menu_index=menulist.index(str(dropdown_menu2.value))
        recommendTime=list()
        recommendTime.append(muldivDawn[region_index][menu_index])
        recommendTime.append(muldivB[region_index][menu_index])
        recommendTime.append(muldivL[region_index][menu_index])
        recommendTime.append(muldivD[region_index][menu_index])
        templist=numpy.array(recommendTime).argsort()
        timelist=["새벽", "아침", "점심", "저녁"]
        nummenu=len(timelist)
        while nummenu!=0:
            print(str(4-nummenu+1)+". "+str(timelist[templist[nummenu-1]])+"\n")
            nummenu-=1
    
dropdown_region2.observe(response2, names='value')
time_buttons2.observe(response2, names="value")
dropdown_menu2.observe(response2, names="value")
print("**********************************************")
print("\n4. 요식업 지역생활권별 업종&영업시간 추천\n")
print('지역생활권을 선택하세요')
dropdown_region2.value='지역생활권을 선택하세요'
time_buttons2.value='0-24'
dropdown_menu2.value='업종을 선택하세요'
display(container1)

**********************************************

4. 요식업 지역생활권별 업종&영업시간 추천

지역생활권을 선택하세요
